# 코틀린 공변성(Covariance)
- 타입 매개변수의 서브타입 관계가 제네릭 타입에도 보존되는 것


In [1]:
fun convertToInt(collection: List<Number>) {
    for (element in collection) {
        println(element.toInt())
    }
}

//List는 공변
val shrtList: List<Short> = listOf(1,2,3,4)
convertToInt(shrtList)

1
2
3
4


In [6]:
//문제상황
fun converToInt2(collection: MutableList<Number>) {
    for (element in collection) {
        println(element.toInt())
    }
}

// MutableList는 무공
val shortList2: MutableList<Short> = mutableListOf(1, 2, 3, 4)
converToInt2(shortList2)

org.jetbrains.kotlinx.jupyter.exceptions.ReplCompilerException: at Cell In[6], line 8, column 14: Type mismatch: inferred type is MutableList<Short> but MutableList<Number> was expected

## 2. Variance(가변성) 핵심 개념

- 무공변(invariant): MutableList<Short> ≠ MutableList<Number>
- 공변(covariant): List<Short> <: List<Number> (out 키워드)
- 반공변(contravariant): Consumer<Number> <: Consumer<Short> (in 키워드)

In [8]:
open class Flower
class Rose : Flower()
class Daisy : Flower()

// 무공변성
class Garden<T : Flower>{
    private val flowers = mutableListOf<T>()

    fun pickFlower(index: Int): T = flowers[index]
//    fun plantFlwer(flower: T): T = flowers.add(flower) 오류
}

fun waterGarden(garden: Garden<Flower>) {
    // 물주기 로직
}

fun tendGarden() {
    val roseGarden = Garden<Rose>()
   // waterGarden(roseGarden) // ❌ 오류: Garden<Rose>를 Garden<Flower>에 전달할 수 없음
}




In [9]:
//공변성
class Garden<out T : Flower>{
    private val flowers = mutableListOf<T>()
    fun pickFlower(index: Int): T = flowers[index] // ✅ 가능 (out position)
    // fun plantFlower(flower: T) // ❌ 불가능 (in position)
}

fun waterGarden(garden: Garden<Flower>) {
    // 물주기 로직
}

fun tendGarden() {
    val roseGarden = Garden<Rose>()
    waterGarden(roseGarden) // ✅ 작동! 서브타입 관계 보존됨
}

## 3. In Position vs Out Position
- Out Position: 함수의 반환 타입 - 데이터를 "내보내는" 위치 : 안전 ✅
- In Position: 함수의 매개변수 타입 - 데이터를 "받아들이는" 위치 : 위험 ❌
#### 제약 이유
- 타입 안정성 보장

In [ ]:
class Container<out T> {
    fun get(): T        // ✅ Out position - 허용
    //fun set(value: T)   // ❌ In position - 금지
}

In [ ]:
// MutableList가 공변이 아닌 이유
fun addNumber(list: MutableList<Number>) {
    list.add(25.5) // Double을 추가
}

val shortList: MutableList<Short> = mutableListOf(1, 2, 3)
addNumber(shortList) // Short 리스트에 Double이 들어감 - 타입 안전성 위반!

#### @UnsafeVariance 주석
읽기 전용 작업에서 안전하다고 확신할 때 컴파일러 경고를 억제

In [ ]:
public interface List<out E> : Collection<E> {
    // contains는 리스트를 변경하지 않으므로 안전
    public operator fun contains(element: @UnsafeVariance E): Boolean
}

| 상황            | 선택         | 이유            |
| ------------- | ---------- | ------------- |
| 읽기 전용 컬렉션     | 공변 (`out`) | 타입 유연성 + 안전   |
| 수정 가능한 컬렉션    | 무공변        | 타입 안전성 보장     |
| 제네릭이 Producer | `out`      | 데이터를 생산(내보냄)  |
| 제네릭이 Consumer | `in`       | 데이터를 소비(받아들임) |


- 공변성(covariance)은 “읽기 전용 Producer”일 때만 적용한다.
- 수정이 들어가면 안전하지 않으니 무공변으로 남겨둬야 한다.